In [60]:
import pandas as pd 
import numpy as np 
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score,f1_score,mean_squared_error, r2_score
from scipy.stats import ttest_1samp, ttest_rel, ttest_ind, chisquare, chi2_contingency, shapiro, bartlett, levene, wilcoxon,f_oneway,kruskal,ranksums

> 문제 1) 다음은 혈압약을 먹은 전, 후의 혈압 데이터이다. 혈압약을 먹기 전, 후의 차이가 있는지 쌍체 t 검정을 실시하시오    
(유의수준5%)   

    * before : 혈압약을 먹기 전 혈압
    * after : 혈압약을 먹은 후의 혈압
    * 귀무가설(H0) : 먹기전, 후가 같다 (after-before =0)
    * 대립가설(H1) : 먹기전, 후가 다르다 (after-before <>0)

In [61]:
# 모집단은 2개 - 동일객체 전후 : ttest_rel / wilcoxon
# 데이터 만들기
df = pd.DataFrame( {
    'before': [120, 135, 122, 124, 135, 122, 145, 160, 155, 142, 144, 135, 167],
    'after' : [110, 132, 123, 119, 123, 115, 140, 162, 142, 138, 135, 142, 160] })

In [62]:
# 정규성 검정 
diff = df['after']-df['before']
n_s, n_p = shapiro(diff)
if n_p >0.05 : 
    print('정규성을 만족한다')
else : 
    print('정규성을 만족 하지 않는다 ')

정규성을 만족한다


In [63]:
# 정규성 만족 -> ttest_rel 
# 정규성 만족 x -> wilcoxon

n_s,n_p = ttest_rel(df['after'],df['before']) # 순서 주의 부호 바뀐다
if n_p >0.05 :
    print('혈압약을 먹기 전 후가 같다')
else : 
    print('혈압약을 먹기 전 후가 다르다')

혈압약을 먹기 전 후가 다르다


In [ ]:
# 만약 정규성을 따르지 않는다면 -> wilcoxon
n_s, n_p =wilcoxon(diff)
if n_p >0.05 : 
    print('혈압약을 먹기 전 후가 같다')
else:
    print('혈압약을 먹기 전 후가 다르다')

혈압약을 먹기


> 문제2) 다음은 A그룹과 B 그룹 인원의 혈압 데이터이다. 두 그룹의 혈압 평균이 다르다고 할 수 있는지 독립표본 t검정을 실시하시오     
(유의수준 5%)

    * A : A 그룹 인원의 혈압
    * B : B 그룹 인원의 혈압
    * H0 (귀무가설) : 두 그룹 혈압이 동일하다 (A=B)
    * H1 (대립가설) : 두 그룹 혈압이 다르다 (A<>B)

In [65]:
df = pd.DataFrame({
  'A' : [120, 135, 122, 124, 135, 122, 145, 160, 155, 142, 144, 135, 167],
  'B' : [110, 132, 123, 119, 123, 115, 140, 162, 142, 138, 135, 142, 160]
})

print(df.head())

     A    B
0  120  110
1  135  132
2  122  123
3  124  119
4  135  123


In [66]:
# 모집단은 2개 - 서로다른 그룹(A,B)간의 비교 : ttest_ind / ranksum
# 독립표본검정 에서는 등분산성 고려 해야한다!!! -> bartlett / levene(정규성 확실하지 않을때)
# A,B 그룹 모두 정규성 만족 -> ttest_ind
# 둘장 하나라도 정규성 만족 x -> ranksum

a_n_s, a_n_p = shapiro(df['A'])
b_n_s, b_n_p = shapiro(df['B'])

if (a_n_p >0.05) & (b_n_p >0.05):
    print('A,B 그룹 둘다 정규성을 만족한다')
else : 
    print('둘중 하나라도 정규성을 만족하지 않는다')



A,B 그룹 둘다 정규성을 만족한다


In [67]:
# 등분산성 확인 
eq_s, eq_p = bartlett(df['A'],df['B'])
if eq_p>0.05 : 
    print('등분산성을 가진다')
else : 
    print('등분산성을 가지지 않는다')

print(eq_p)

등분산성을 가진다
0.867271716287991


In [ ]:
# equal_var = True : 등분산
# equal_var = False : 등분산 x
s, p = ttest_ind(df['A'],df['B'],equal_var=True,alternative='two-sided')
if p >0.05 : 
    print('두 그룹 혈압이 동일하다')
else: 
    print('두 그룹 혈압이 다르다')
print(p)

두 그룹 혈압이 동일하다
0.4207486213941666


In [71]:
# 정규성을 만족 하지 않는 경우 비모수 검정 (ranksums)
r_s, r_p = ranksums(df['A'],df['B'],alternative='two-sided')
print(r_p)

0.39746692542259277


> 문제 2) 다음은 A그룹과 B그룹 인원의 혈압 데이터이다.    
A그룹의 혈압 평균이 B그룹보다 크다고 할 수 있는지 독립표본 t검정을 실시하시오 (단측검정)    

    * 귀무가설(H0) : A 그룹 혈압 평균 <= B 그룹 혈압 평균
    * 대립가설(H1) : A 그룹 혈압형균 > B 그룹 혈압 평균

In [72]:
# 모집단 2개 독립검정 -> ttest_ind /ranksums
# 정규성 검정 : shapiro
# 등분산 검정 : bartlett / levene

# 두 그룹 모두 정규성 만족 & 등분산성 만족시 -> ttest_ind (equal_var=True)
# 하나라도 정규성 만족 하지 않을때 -> ranksums

a_n_s, a_n_p = shapiro(df['A'])
b_n_s, b_n_p = shapiro(df['B'])

if (a_n_s >0.05)&(b_n_s >0.05) :
    print('두 그룹 모두 정규성을 만족한다')
else :
    print('두 그룹중 하나라도 정규성을 만족하지 않는다')

두 그룹 모두 정규성을 만족한다


In [73]:
b_s,b_p = bartlett(df['A'],df['B'])

if b_p >0.05 : 
    print('등분산성을 만족한다')
else : 
    print('등분산성을 만족하지 않는다')


등분산성을 만족한다


In [75]:
n_s, n_p = ttest_ind(df['A'],df['B'],equal_var=True, alternative='greater')

if n_p >0.05 : 
    print('A 그룹 혈압 평균이  B 그룹 혈압평균보다 작거나 같다')
else : 
    print('A 그룹 혈압 평균이 B 그룹 혈압 평균보다 크다')

A 그룹 혈압 평균이  B 그룹 혈압평균보다 작거나 같다


In [76]:
# 만약 정규성을 만족하지 않는 경우 -> ranksums 순위합 검정 실시
x_s, x_p = ranksums(df['A'],df['B'],alternative='greater')

if x_p >0.05:
    print('A 그룹 혈압 평균이  B 그룹 혈압평균보다 작거나 같다')
else : 
    print('A 그룹 혈압 평균이 B 그룹 혈압 평균보다 크다')

A 그룹 혈압 평균이  B 그룹 혈압평균보다 작거나 같다
